In [1]:
import numpy as np

In [2]:
def tanh(z):
    """
    Implements the tanh function.

    Parameters
    ----------
    z: (ndarray of any shape) or (scalar) - input to the tanh function

    Returns
    -------
    a: (ndarray of same shape as Z) or (scalar) - output from the tanh function
    """

    a = (np.exp(z) - np.exp(-z)) / (np.exp(z) + np.exp(-z))
    return a


def softmax(z):
    """
    Implements the softmax activation.

    Parameters
    ----------
    z: (ndarray of any shape) - input to the activation function

    Returns
    -------
    a: (ndarray of same shape as Z) - output of the activation function
    """

    # Subtracting the maximum value in each column for numerical stability to avoid overflow
    z_stable = z - np.max(z, axis=0, keepdims=True)
    exp_z = np.exp(z_stable)
    a = exp_z / np.sum(exp_z, axis=0, keepdims=True)
    return a

In [3]:
def rnn_cell_forward(xt, at_prev, parameters):
    """
    Implements a single forward step of the RNN-cell.

    Parameters
    ----------
    xt: (ndarray (n_x, m)) - input data at timestep "t"
    at_prev: (ndarray (n_a, m)) - hidden state at timestep "t-1"
    parameters:
        Waa: (ndarray (n_a, n_a)) - weight matrix multiplying the hidden state at_prev
        Wax: (ndarray (n_a, n_x)) - weight matrix multiplying the input xt
        Wya: (ndarray (n_y, n_a)) - weight matrix relating the hidden-state to the output
        ba: (ndarray (n_a, 1)) - bias
        by: (ndarray (n_y, 1)) - bias

    Returns
    -------
    at: (ndarray (n_a, m)) - hidden state at timestep "t"
    yt: (ndarray (n_y, m)) - prediction at timestep "t"
    cache: (tuple) - returning (at, at_prev, xt, zxt, y_hat_t, zyt) for the backpropagation
    """

    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    ba = parameters["ba"]
    Wyx = parameters["Wya"]
    by = parameters["by"]

    zxt = Waa @ at_prev + Wax @ xt + ba
    at = np.tanh(zxt)
    zyt = Wyx @ at + by
    y_hat_t = softmax(zyt)

    cache = (at, at_prev, xt, zxt, y_hat_t, zyt)
    return at, y_hat_t, cache


def rnn_forward(X, a0, parameters):
    """
    Implements the forward propagation of the RNN.

    Parameters
    ----------
    X: (ndarray (n_x, m, T_x)) - input data for each timestep
    a0: (ndarray (n_a, m)) - initial hidden state
    parameters:
        Waa: (ndarray (n_a, n_a)) - weight matrix multiplying the hidden state at_prev
        Wax: (ndarray (n_a, n_x)) - weight matrix multiplying the input xt
        Wyx: (ndarray (n_y, n_a)) - weight matrix relating the hidden-state to the output
        ba: (ndarray (n_a, 1)) - bias
        by: (ndarray (n_y, 1)) - bias

    Returns
    -------
    a: (ndarray (n_a, m, T_x)) - hidden states for each timestep
    y_hat: (ndarray (n_y, m, T_x)) - predictions for each timestep
    caches: (tuple) - returning (list of cache, x) for the backpropagation
    """

    caches = []
    n_x, m, T_x = X.shape
    n_y, n_a = parameters["Wya"].shape

    a = np.zeros((n_a, m, T_x))
    y_hat = np.zeros((n_y, m, T_x))

    at_prev = a0
    for t in range(T_x):
        at_prev, y_hat_t, cache = rnn_cell_forward(X[:, :, t], at_prev, parameters)
        a[:, :, t] = at_prev
        y_hat[:, :, t] = y_hat_t
        caches.append(cache)

    return a, y_hat, caches

In [4]:
def rnn_cell_backward(y, dat, cache, parameters):
    """
    Implements a single backward step of the RNN-cell.

    Parameters
    ----------
    y: (ndarray (n_y, m)) - true labels at timestep "t"
    dat: (ndarray (n_a, m)) - gradient of the hidden state at timestep "t"
    cache: (tuple) - (at, at_prev, xt, zxt, y_hat_t, zyt)
    parameters:
        Waa: (ndarray (n_a, n_a)) - weight matrix multiplying the hidden state at_prev
        Wax: (ndarray (n_a, n_x)) - weight matrix multiplying the input xt
        Wya: (ndarray (n_y, n_a)) - weight matrix relating the hidden-state to the output
        ba: (ndarray (n_a, 1)) - bias
        by: (ndarray (n_y, 1)) - bias

    Returns
    -------
    gradients: (dict) - the gradients
        dWaa: (ndarray (n_a, n_a)) - gradient of the weight matrix multiplying the hidden state at_prev
        dWax: (ndarray (n_a, n_x)) - gradient of the weight matrix multiplying the input xt
        dWya: (ndarray (n_y, n_a)) - gradient of the weight matrix relating the hidden-state to the output
        dba: (ndarray (n_a, 1)) - gradient of the bias
        dby: (ndarray (n_y, 1)) - gradient of the bias
        dat: (ndarray (n_a, m)) - gradient of the hidden state
    """

    at, at_prev, xt, zt, y_hat_t, zyt = cache
    dy = y_hat_t - y
    gradients = {
        "dWya": dy @ at.T,
        "dby": np.sum(dy, axis=1, keepdims=True),
    }
    da = parameters["Wya"].T @ dy + dat
    dz = (1 - at ** 2) * da
    gradients["dba"] = np.sum(dz, axis=1, keepdims=True)
    gradients["dWax"] = dz @ xt.T
    gradients["dWaa"] = dz @ at_prev.T
    gradients["dat"] = parameters["Waa"].T @ dz
    return gradients


def rnn_backward(X, Y, parameters, caches):
    """
    Implements the backward propagation of the RNN.

    Parameters
    ----------
    X: (ndarray (n_x, m, T_x)) - input data
    Y: (ndarray (n_y, m, T_x)) - true labels
    parameters:
        Waa: (ndarray (n_a, n_a)) - weight matrix multiplying the hidden state at_prev
        Wax: (ndarray (n_a, n_x)) - weight matrix multiplying the input xt
        Wya: (ndarray (n_y, n_a)) - weight matrix relating the hidden-state to the output
        ba: (ndarray (n_a, 1)) - bias
        by: (ndarray (n_y, 1)) - bias
    caches: (list) - list of caches from rnn_forward

    Returns
    -------
    gradients: (dict) - the gradients
        dWaa: (ndarray (n_a, n_a)) - gradient of the weight matrix multiplying the hidden state at_prev
        dWax: (ndarray (n_a, n_x)) - gradient of the weight matrix multiplying the input xt
        dWya: (ndarray (n_y, n_a)) - gradient of the weight matrix relating the hidden-state to the output
        dba: (ndarray (n_a, 1)) - gradient of the bias
        dby: (ndarray (n_y, 1)) - gradient of the bias
        dat: (ndarray (n_a, m)) - gradient of the hidden state
    """

    n_x, m, T_x = X.shape
    a1, a0, x1, zx1, y_hat1, zy1 = caches[0]
    Waa, Wax, Wya, by, ba = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['ba']

    gradients = {
        "dWaa": np.zeros_like(Waa), "dWax": np.zeros_like(Wax), "dWya": np.zeros_like(Wya),
        "dba": np.zeros_like(ba), "dby": np.zeros_like(by), "dat": np.zeros_like(a0),
    }

    dat = np.zeros_like(a0)
    for t in reversed(range(T_x)):
        grads = rnn_cell_backward(Y[:, :, t], dat, caches[t], parameters)
        for key in gradients:
            gradients[key] += grads[key]
        dat = grads["dat"]

    return gradients

In [5]:
def compute_loss(Y_hat, Y):
    """
    Computes the cross-entropy loss.

    Parameters
    ----------
    Y_hat: (ndarray (n_y, m, T_x)) - predictions for each timestep
    Y: (ndarray (n_y, m, T_x)) - true labels

    Returns
    -------
    loss: (float) - the cross-entropy loss
    """

    return -np.sum(Y * np.log(Y_hat))

In [6]:
def clip(gradients, max_value):
    """
    Clips the gradients to a maximum value.

    Parameters
    ----------
    gradients: (dict) - the gradients
        dWaa: (ndarray (n_a, n_a)) - gradient of the weight matrix multiplying the hidden state at_prev
        dWax: (ndarray (n_a, n_x)) - gradient of the weight matrix multiplying the input xt
        dWya: (ndarray (n_y, n_a)) - gradient of the weight matrix relating the hidden-state to the output
        dba: (ndarray (n_a, 1)) - gradient of the bias
        dby: (ndarray (n_y, 1)) - gradient of the bias
    max_value: (float) - the maximum value to clip the gradients

    Returns
    -------
    gradients: (dict) - the clipped gradients
    """

    dWaa, dWax, dWya = gradients["dWaa"], gradients["dWax"], gradients["dWya"]
    dba, dby = gradients["dba"], gradients["dby"]

    for gradient in [dWax, dWaa, dWya, dba, dby]:
        np.clip(gradient, -max_value, max_value, out=gradient)

    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "dba": dba, "dby": dby}
    return gradients


def update_parameters(parameters, gradients, learning_rate):
    """
    Updates the parameters using the gradients.

    Parameters
    ----------
    parameters: (dict) - the parameters
        Waa: (ndarray (n_a, n_a)) - weight matrix multiplying the hidden state at_prev
        Wax: (ndarray (n_a, n_x)) - weight matrix multiplying the input xt
        Wya: (ndarray (n_y, n_a)) - weight matrix relating the hidden-state to the output
        ba: (ndarray (n_a, 1)) - bias
        by: (ndarray (n_y, 1)) - bias
    gradients: (dict) - the gradients
        dWaa: (ndarray (n_a, n_a)) - gradient of the weight matrix multiplying the hidden state at_prev
        dWax: (ndarray (n_a, n_x)) - gradient of the weight matrix multiplying the input xt
        dWya: (ndarray (n_y, n_a)) - gradient of the weight matrix relating the hidden-state to the output
        dba: (ndarray (n_a, 1)) - gradient of the bias
        dby: (ndarray (n_y, 1)) - gradient of the bias
    learning_rate: (float) - the learning rate
    """

    parameters["Waa"] -= learning_rate * gradients["dWaa"]
    parameters["Wax"] -= learning_rate * gradients["dWax"]
    parameters["Wya"] -= learning_rate * gradients["dWya"]
    parameters["ba"] -= learning_rate * gradients["dba"]
    parameters["by"] -= learning_rate * gradients["dby"]

In [7]:
def optimize(X, Y, a_prev, parameters, learning_rate):
    """
    Implements the forward and backward propagation of the RNN.

    Parameters
    ----------
    X: (ndarray (n_x, m, T_x)) - the input data
    Y: (ndarray (n_y, m, T_x)) - true labels
    a_prev: (ndarray (n_a, m)) - the initial hidden state
    parameters: (dict) - the initial parameters
        Waa: (ndarray (n_a, n_a)) - weight matrix multiplying the hidden state A_prev
        Wax: (ndarray (n_a, n_x)) - weight matrix multiplying the input Xt
        Wya: (ndarray (n_y, n_a)) - weight matrix relating the hidden-state to the output
        ba: (ndarray (n_a, 1)) - bias
        by: (ndarray (n_y, 1)) - bias
    learning_rate: (float) - the learning rate

    Returns
    -------
    loss: (float) - the cross-entropy loss
    gradients: (dict) - the gradients
    a: (ndarray (n_a, m)) - the hidden state at the last timestep
    """

    a, Y_hat, caches = rnn_forward(X, a_prev, parameters)
    loss = compute_loss(Y_hat, Y)
    gradients = rnn_backward(X, Y, parameters, caches)
    gradients = clip(gradients, 5)
    update_parameters(parameters, gradients, learning_rate)
    return loss, gradients, a[:, :, -1]

In [8]:
def initialize_parameters(n_a, n_x, n_y):
    """
    Initializes the parameters for the RNN.

    Parameters
    ----------
    n_a: (int) - number of units in the hidden state
    n_x: (int) - number of units in the input data
    n_y: (int) - number of units in the output data

    Returns
    -------
    parameters: (dict) - initialized parameters
        "Wax": (ndarray (n_a, n_x)) - weight matrix multiplying the input
        "Waa": (ndarray (n_a, n_a)) - weight matrix multiplying the hidden state
        "Wya": (ndarray (n_y, n_a)) - weight matrix relating the hidden-state to the output
        "ba": (ndarray (n_a, 1)) - bias
        "by": (ndarray (n_y, 1)) - bias
    """

    Wax = np.random.randn(n_a, n_x) * 0.01
    Waa = np.random.randn(n_a, n_a) * 0.01
    Wya = np.random.randn(n_y, n_a) * 0.01
    ba = np.zeros((n_a, 1))
    by = np.zeros((n_y, 1))
    parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "ba": ba, "by": by}
    return parameters

In [9]:
chars = "\nabcdefghijklmnopqrstuvwxyz"
char_to_index = {ch: i for i, ch in enumerate(chars)}
index_to_char = {i: ch for i, ch in enumerate(chars)}


def rnn_model(inputs, num_iterations, learning_rate):
    m = 1
    n_a = 50
    n_x = n_y = len(chars)
    a_prev = np.zeros((n_a, m))
    parameters = initialize_parameters(n_a, n_x, n_y)

    for j in range(num_iterations):
        index = j % len(inputs)
        example = inputs[index]
        example_indexes = [char_to_index[c] for c in example]

        T_x = len(example_indexes) + 1
        X = np.zeros((n_x, m, T_x))
        Y = np.zeros((n_y, m, T_x))
        for i, idx in enumerate([None] + example_indexes):
            if idx:
                X[idx, m - 1, i] = 1
        for i, idx in enumerate(example_indexes + [char_to_index["\n"]]):
            Y[idx, m - 1, i] = 1

        loss, gradients, a_prev = optimize(X, Y, a_prev, parameters, learning_rate)

    return parameters

In [10]:
def sample(parameters, char_to_ix, ix_to_char):
    n_x = parameters["by"].shape[0]
    n_a = parameters["Waa"].shape[1]
    x = np.zeros((n_x, 1))
    a_prev = np.zeros((n_a, 1))

    indices = []
    idx = -1
    newline_char = char_to_ix["\n"]

    while idx != newline_char and len(indices) != 50:
        a_prev, y_hat, cache = rnn_cell_forward(x, a_prev, parameters)
        idx = np.random.choice(list(range(n_x)), p=y_hat.ravel())
        indices.append(idx)
        x = np.zeros((n_x, 1))
        x[idx] = 1

    if indices[-1] != newline_char:
        indices.append(char_to_ix["\n"])

    text = ''.join(ix_to_char[ix] for ix in indices)
    return text

In [11]:
data = open("dinos.txt", "r").read().split("\n")
data = [x.strip().lower() for x in data]
parameters = rnn_model(data, 22001, 0.01)
text = sample(parameters, char_to_index, index_to_char)
text

'werrosaurus\n'